In [ ]:
import findspark
findspark.init()
from glob import glob
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils
conf = SparkConf()
conf.setMaster("loacal[*]")
conf.set("spark.executor.memory", "16g")
sc = SparkContext(conf=conf)


In [ ]:
spamlist=list(map(lambda p: sc.textFile(p),glob('chs_cut/trec06c/spam/*/*')))

hamlist=list(map(lambda p: sc.textFile(p),glob('chs_cut/trec06c/ham/*/*')))
#spamlist=list(map(lambda p: sc.textFile(p),glob('chs_cut/trec06c/spamtest/*')))
#hamlist=list(map(lambda p: sc.textFile(p),glob('chs_cut/trec06c/hamtest/*')))
spam=sc.union(spamlist)
ham=sc.union(hamlist)


        

In [ ]:
# Create a HashingTF instance to map sms text to vectors of 100 features.
tf = HashingTF(numFeatures = 100)
# Each messages is split into words, and each word is mapped to one feature.
spamFeatures = spam.map(lambda msg: tf.transform(msg.split(" ")))
hamFeatures = ham.map(lambda msg: tf.transform(msg.split(" ")))

In [ ]:
# Create LabeledPoint datasets for positive (spam) and negative (ham) examples.
positiveExamples = spamFeatures.map(lambda features: LabeledPoint(1, features))
negativeExamples = hamFeatures.map(lambda features: LabeledPoint(0, features))
training_data = positiveExamples.union(negativeExamples)
# cache the training data
training_data.cache()

In [ ]:
#create training, test sets
trainset, testset = training_data.randomSplit([0.6, 0.4])

In [ ]:
#Fit NaiveBayes
model = NaiveBayes.train(trainset, 1.0)

In [ ]:
#predict
predictionLabel = testset.map(lambda x: (model.predict(x.features), x.label))

In [ ]:
#model accuracy

accuracy = 1.0 * predictionLabel.filter(lambda x : x[0]==x[1]).count()/testset.count()
print ("Model accuracy : {:.2f}".format(accuracy))

In [ ]:
print(glob('chs_cut/trec06c/spam/*/*'))

In [ ]:
def getFeatures(sc,tf,list) {
        l = list.parallelStream().flatMap(path -> {
            try {
                return Files.lines(path, StandardCharsets.UTF_8).parallel().map(s -> tf.transform(Arrays.asList(s.split(SP_CHAR))));
            } catch (IOException e) {
                e.printStackTrace();
                // return new LinkedList<>().stream();
                return null;
            }
        }).collect(Collectors.toList());

        return sc.parallelize(l);
    }